In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import os
import json
import torch.nn as nn
import math
from tqdm.auto import tqdm

/home/abhijeet/miniconda3/envs/TRIZ/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

0 NVIDIA GeForce RTX 3090


In [3]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [4]:
batch_size = 16
n_epochs = 500
model_no = 'transformer_with_word2vec'
exp = 1

In [5]:
# With square kernels and equal stride
# m = nn.Conv2d(5500, 5500, 1, stride=1)
# non-square kernels and unequal stride and with padding
m = nn.Conv1d(5500, 5500, 1, stride=1)
# # non-square kernels and unequal stride and with padding and dilation
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 5500, 1)
output = m(input)
print(output.shape)

torch.Size([20, 5500, 1])


In [6]:
class custom_transformer(nn.Module):

    def __init__(self, hidden_dim=100, nheads=5, num_encoder_layers=5, num_decoder_layers=5, avg_words = 5500):
        super(custom_transformer, self).__init__()
#         self.transformer = nn.Transformer(hidden_dim, nheads, num_encoder_layers, num_decoder_layers, 
#                                           batch_first=True, activation="relu")
        self.encoder = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nheads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder, num_layers = num_encoder_layers)
        
        self.decoder = nn.TransformerDecoderLayer(d_model=hidden_dim, nhead=nheads, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder, num_layers = num_decoder_layers)
        
#         self.linear1 = nn.Linear(hidden_dim*avg_words, hidden_dim*avg_words, bias=True)
#         self.one_D_conv = nn.Conv2d(hidden_dim*avg_words, hidden_dim*avg_words, 1, stride=1)
        self.relu_layer = nn.ReLU()
        self.sigmoid_layer = nn.Sigmoid()
        
    def positionalencoding1d(self, d_model, length):
        """
        :param d_model: dimension of the model
        :param length: length of positions
        :return: length*d_model position matrix
        """
        if d_model % 2 != 0:
            raise ValueError("Cannot use sin/cos positional encoding with "
                             "odd dim (got dim={:d})".format(d_model))
        pe = torch.zeros(length, d_model)
        position = torch.arange(0, length).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                             -(math.log(10000.0) / d_model)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)

        return pe

    def forward(self, feat_input):
        feat_input = feat_input.flatten(2)
#         print(feat_input.shape)
#         print(self.positionalencoding1d(self.hidden_dim, feat_input.shape[-2]).repeat(feat_input.shape[0], feat_input.shape[1], feat_input.shape[2]).is_cuda)
#         feat_input += self.positionalencoding1d(self.hidden_dim, feat_input.shape[-2]).repeat(feat_input.shape[0], feat_input.shape[1], feat_input.shape[2])
#         features = self.transformer(feat_input.cuda(), self.learnable_query.repeat(feat_input.shape[0], 1, 1))
        enc_features = self.transformer_encoder(feat_input)
            
        dec_features = self.transformer_decoder(feat_input, enc_features)
#         a.unsqueeze(2)
#         print(dec_features.flatten(1).shape)
#         print(dec_features.flatten(1).unsqueeze(2).shape)
#         features = self.linear1(dec_features.flatten(1))
        dec_features = self.sigmoid_layer(features)

        return dec_features

In [7]:
new_model = custom_transformer()
# new_model.to(device)
print("Total trainable params:", torch.nn.utils.parameters_to_vector([p for p in new_model.parameters() if p.requires_grad]).numel())

Total trainable params: 5674176


/home/abhijeet/miniconda3/envs/TRIZ/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
new_model

custom_transformer(
  (encoder): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-4): 5 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
    

In [9]:
# Keeping Stop Words

In [10]:
from torch.utils.data import Dataset, DataLoader

class dataset_loader(Dataset):
    
    def __init__(self, corpus_dir, word2vec_dir):
        self.word2vec_all = open(word2vec_dir, 'r')
        self.word2vec_all = json.load(self.word2vec_all)
        self.dataset = pd.read_excel(corpus_dir + 'generated_data.xlsx')
        self.dataset = self.dataset.dropna()
        self.text = self.dataset['text']
        self.label = self.dataset['labels']
        
    def __len__(self):
        return self.text.shape[0]
    
    def __getitem__(self, idx):
        row_text = self.text[idx].lower().replace('\n', ' ').split()
        row_label = eval(self.label[idx])
#         print(row_label.shape, row_text.shape)
        word2vec_matrix = []
        
        for count, i in enumerate(row_text):
            try:
                word2vec_matrix.append(np.array(self.word2vec_all[i]))
                if count == 5000:
                    break
            except:
                word2vec_matrix.append(np.zeros(100))
        
        for i in range(5000 - len(word2vec_matrix)):
            word2vec_matrix.append(np.zeros(100))
            
#         print(len(word2vec_matrix))

        for next_count, i in enumerate(row_label):
            try:
                word2vec_matrix.append(np.array(self.word2vec_all[i]))
                if next_count == 499:
                    break
            except:
                word2vec_matrix.append(np.zeros(100))
            
        for i in range(499 - next_count):
            word2vec_matrix.append(np.zeros(100))

#         print(len(word2vec_matrix))
        
        output = {'text_label': np.array(word2vec_matrix)}
        
        return output

In [11]:
train_data = dataset_loader(corpus_dir='/home/abhijeet/Desktop/TRIZ/All_data/CPC Data/',
                              word2vec_dir='/home/abhijeet/Desktop/TRIZ/word_vectors.json')

In [12]:
# print(len(train_data))
for i in range(len(train_data)):
    sample = train_data[i]
    break

In [13]:
batch_size = 1

In [14]:
train_sampler = torch.utils.data.RandomSampler(train_data)

dataloader_train = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=0)

In [15]:
import torch.nn.functional as F
def criterion(predicted, target):
    """
    Compute the Kullback-Leibler Divergence loss between two probability distributions.

    Args:
        p (torch.Tensor): True distribution (e.g., ground truth probabilities).
        q (torch.Tensor): Approximate distribution (e.g., predicted probabilities).

    Returns:
        torch.Tensor: KL Divergence loss.
    """
#     return F.kl_div(F.log_softmax(target, dim=1), F.softmax(predicted, dim=1), reduction='batchmean')
    return F.kl_div(predicted, target)

In [16]:
new_model.to(device)
optimizer = torch.optim.SGD(new_model.parameters(), lr=0.009, weight_decay=0.0001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [17]:
for epoch in range(n_epochs):
    num_nans = 0
    running_loss = 0.0
    val_loss = 0.0
    print(f"\nEpoch: {epoch+1}")
    inner_pbar = tqdm(total=len(dataloader_train), position=1, leave=False, ascii=True, desc=f"Epoch: {epoch+1} Step")

    for i, data in enumerate(dataloader_train, 1):
        features = data['text_label']
        features = features.type(torch.FloatTensor)
        features = features.to(device)
#         print(features.is_cuda)
        y_ground_truth = features
        optimizer.zero_grad()
        print(features.shape)
        preds = new_model(features)
        
        loss = criterion(predicted=preds, target=y_ground_truth)

        inner_pbar.update(1)
        
        running_loss += loss.detach().item()

        print(f"\rStep: {i} Training Loss: {loss.item()} Validation Loss: {val_loss} Epoch loss: {running_loss/(i)}", end="")
        # print(f"\rStep: {i} Training Loss: {loss.item()} Validation Loss: {val_loss} Nans: {num_nans}")

        if torch.isnan(loss):
            if num_nans > 10:
                raise RuntimeError(f"Model Error: Encountered {num_nans} nan loss")
            num_nans += 1
            continue
        # loss.requires_grad = True
        loss.backward()

        optimizer.step()
    scheduler.step()
    inner_pbar.close()


Epoch: 1



Epoch: 1 Step:   0%|                                          | 0/173749 [00:00<?, ?it/s]/home/abhijeet/miniconda3/envs/TRIZ/lib/python3.8/site-packages/torch/nn/functional.py:2949: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(

Epoch: 1 Step:   0%|                                | 1/173749 [00:00<8:00:04,  6.03it/s]

torch.Size([1, 5500, 100])
Step: 1 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])



Epoch: 1 Step:   0%|                                | 5/173749 [00:00<4:07:11, 11.71it/s]

Step: 2 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 3 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 4 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])



Epoch: 1 Step:   0%|                                | 7/173749 [00:00<4:03:50, 11.87it/s]

Step: 5 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 6 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 7 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])



Epoch: 1 Step:   0%|                               | 11/173749 [00:00<4:01:16, 12.00it/s]

Step: 8 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 9 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 10 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 11 Training Loss: nan Validation Loss: 0.0 Epoch loss: nantorch.Size([1, 5500, 100])
Step: 12 Training Loss: nan Validation Loss: 0.0 Epoch loss: nan

RuntimeError: Model Error: Encountered 11 nan loss

In [ ]:
torch.__version__

In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
memory = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_decoder(tgt, memory)


Epoch: 1 Step:   0%|                               | 12/173749 [00:19<4:01:16, 12.00it/s]